# TP1 - NER

### En utilisant vos connaissances en programmation, codez un système NER qui reconnaît les noms de personnes (PERSON) et les lieux (LOCATION) à partir de l'ensemble de données fourni (europarl).

#### 1 - On récupère toutes nos données

In [1]:
import os
import glob
import json
import re
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            fullPath = fullPath.rsplit('/', 1)[-1]
            allFiles.append(fullPath)
                
    return allFiles

def readFile(filename):
    with open(filename, "r", encoding="utf8") as f:
        return f.read()

files = getListOfFiles('europarl')

[nltk_data] Downloading package stopwords to D:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### 2 - Division du texte en phrases et en mots (c'est-à-dire en jetons)

In [2]:
# 1 - On enlève la ponctuation
def enlever_ponctuation(chaine):
    return re.sub(r"[^\w\s]", " ", chaine)

# 2 - On enlève les mots si ils sont dans la liste des mots vides
def enlever_mots_vides(chaine):
    return ' '.join([mot for mot in chaine.split() if mot not in stopwords.words("french")])

# 3 - Tokenisation des mots, on les sépare en mots
def tokenisation(chaine):
    return chaine.split()

# On crée une fonction qui va appliquer les 3 fonctions précédentes
def preprocess(chaine):
    return tokenisation(enlever_mots_vides(enlever_ponctuation(chaine)))

### 3 - Repérer les localisations et les personnes
Résultat :
Les phrases sont séparées par une nouvelle ligne, et chaque ligne contient un mot et son libellé : O s'il n'est pas contenu dans une entité, B-PER s'il fait partie d'une entité et qu'il apparaît au début du nom, et I-PER s'il se trouve à l'intérieur de l'entité ou à la fin de l'entité.


In [20]:
import requests

LOCATIONS = "https://nominatim.openstreetmap.org"

# On interroge une API qui contient toutes les localisations du monde

def est_localisation(mot):
    if mot[0].isupper() == False:
        return False
    response = requests.get(LOCATIONS + "/search?q="+ mot + "&format=json")
    if response.status_code == 200:
        for data in response.json():
            if data["importance"] > 0.5:
                return True
            return False    
    return False


False

In [ ]:
locations = ['Londres', 'Paris']
persons = ['Elie', 'Jean']

# API that stores every firstnames and lastnames of the world
NAMES = "https://api.genderize.io"

# Ask the api if "Adrien" is a name
def est_nom(mot):
    response = requests.get(NAMES + "?name=" + mot +  "&format=json")
    if response.status_code == 200:
        for data in response.json():
            if data["count"] > 0:
                return True
            return False
    return False




# Name Entity recognition (NER)
# Algorithm for a name entity recognition
# Input: a list of words
# Output: a list of tuples (word, tag)
def ner(words):
    tags = []
    for word in words:
        if word in locations:
            tags.append((word, " B-LOC"))
        elif word in persons:
            if len(tags) > 0 and tags[-1][1] == " I-PER":
                tags.append((word, " I-PER"))
            tags.append((word, "B-PER"))
        else:
            tags.append((word, "O"))
    return tags